In [96]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import json
from datetime import datetime
import csv
import sys

In [112]:
recipe_table = pd.read_csv('data/recipe_table.csv')

In [113]:
recipe_table

,Unnamed: 0,BaseGearScore,category,cooldown,description,event,icon,id,ingredients,itemType,name,output,perkBuckets,rarity,recipeLevel,salvageRecipe,stations,tradeskill,type
0,0,NaN,Furniture,NaN,Crafting recipe for Major Fishing Gathering Tr...,"{'CategoricalProgressionId': 'Furnishing', 'Ca...",House_HousingItem_Buff_Fishing_Bonus_T3,house_housingitem_buff_fishing_bonus_t3,"[[""timbert5"", ""Ironwood Planks"", 25], [""ingott...",HousingItem,Recipe: Major Fishing Gathering Trophy,house_housingitem_buff_fishing_bonus_t3,[],3,200,NaN,['Workshop Tier 5'],Furnishing,recipe
1,1,NaN,Furniture,NaN,Crafting recipe for Major Harvesting Gathering...,"{'CategoricalProgressionId': 'Furnishing', 'Ca...",House_HousingItem_Buff_Harvesting_Bonus_T3,house_housingitem_buff_harvesting_bonus_t3,"[[""timbert5"", ""Ironwood Planks"", 25], [""ingott...",HousingItem,Recipe: Major Harvesting Gathering Trophy,house_housingitem_buff_harvesting_bonus_t3,[],3,200,NaN,['Workshop Tier 5'],Furnishing,recipe
2,2,NaN,Furniture,NaN,Crafting recipe for Major Logging Gathering Tr...,"{'CategoricalProgressionId': 'Furnishing', 'Ca...",House_HousingItem_Buff_Logging_Bonus_T3,house_housingitem_buff_logging_bonus_t3,"[[""timbert5"", ""Ironwood Planks"", 25], [""ingott...",HousingItem,Recipe: Major Logging Gathering Trophy,house_housingitem_buff_logging_bonus_t3,[],3,200,NaN,['Workshop Tier 5'],Furnishing,recipe
3,3,NaN,Furniture,NaN,Crafting recipe for Major Loot Luck Trophy,"{'CategoricalProgressionId': 'Furnishing', 'Ca...",House_HousingItem_Buff_Luck_Bonus_T3,house_housingitem_buff_luck_bonus_t3,"[[""timbert5"", ""Ironwood Planks"", 25], [""ingott...",HousingItem,Recipe: Major Loot Luck Trophy,house_housingitem_buff_luck_bonus_t3,[],4,200,NaN,['Workshop Tier 5'],Furnishing,recipe
4,4,NaN,Furniture,NaN,Crafting recipe for Major Lost Combat Trophy,"{'CategoricalProgressionId': 'Furnishing', 'Ca...",House_HousingItem_Buff_Damage_Vs_Lost_T3,house_housingitem_buff_damage_vs_lost_t3,"[[""timbert5"", ""Ironwood Planks"", 25], [""ingott...",HousingItem,Recipe: Major Lost Combat Trophy,house_housingitem_buff_damage_vs_lost_t3,[],3,200,NaN,['Workshop Tier 5'],Furnishing,recipe
5,5,NaN,Furniture,NaN,Crafting recipe for Major Cooking Crafting Trophy,"{'CategoricalProgressionId': 'Furnishing', 'Ca...",House_HousingItem_Buff_Cooking_Bonus_T3,house_housingitem_buff_cooking_bonus_t3,"[[""timbert5"", ""Ironwood Planks"", 25], [""ingott...",HousingItem,Recipe: Major Cooking Crafting Trophy,house_housingitem_buff_cooking_bonus_t3,[],3,200,NaN,['Workshop Tier 5'],Furnishing,recipe
6,6,NaN,Furniture,NaN,Crafting recipe for Major Corrupted Combat Trophy,"{'CategoricalProgressionId': 'Furnishing', 'Ca...",House_HousingItem_Buff_Damage_Vs_Corrupted_T3,house_housingitem_buff_damage_vs_corrupted_t3,"[[""timbert5"", ""Ironwood Planks"", 25], [""ingott...",HousingItem,Recipe: Major Corrupted Combat Trophy,house_housingitem_buff_damage_vs_corrupted_t3,[],3,200,NaN,['Workshop Tier 5'],Furnishing,recipe
7,7,NaN,Furniture,NaN,Crafting recipe for Major Engineering Crafting...,"{'CategoricalProgressionId': 'Furnishing', 'Ca...",House_HousingItem_Buff_Engineering_Bonus_T3,house_housingitem_buff_engineering_bonus_t3,"[[""timbert5"", ""Ironwood Planks"", 25], [""ingott...",HousingItem,Recipe: Major Engineering Crafting Trophy,house_housingitem_buff_engineering_bonus_t3,[],3,200,NaN,['Workshop Tier 5'],Furnishing,recipe
8,8,NaN,Furniture,NaN,Crafting recipe for Major Mining Gathering Trophy,"{'CategoricalProgressionId': 'Furnishing', 'Ca...",House_HousingItem_Buff_Mining_Bonus_T3,house_housingitem_buff_mining_bonus_t3,"[[""timbert5"", ""Ironwood Planks"", 25], [""ingott...",HousingItem,Recipe: Major Mining Gathering Trophy,house_housingitem_buff_mining_bonus_t3,[],3,200,NaN,['Workshop Tier 5'],Furnishing,recipe
9,9,NaN,Furniture,NaN,Crafting recipe for Major Skinning Gathering T...,"{'CategoricalProgressionId': 'Furnishing', 'Ca...",House_HousingItem_Buff_Skinning_Bonus_T3,house_housingitem_buff_skinn

In [114]:
recipe_table.iloc[0]['ingredients']

'[["timbert5", "Ironwood Planks", 25], ["ingott5", "Orichalcum Ingot", 20], ["woodstaint5", "Mahogany Stain", 1], ["house_housingitem_fishing_veryrarefish_c1", "Taxidermied Daemonaja", 1], ["house_housingitem_buff_fishing_bonus_t2", "Basic Fishing Gathering Trophy", 1]]'

In [119]:
def createOverlapDict(table):
    """
    It creates a dictionary of the form {item1: {item2: # resources to make item 1, item3: # resources to make item 1}}
    {ironArrow: {ironIngot: 4, lumber: 2, feathers: 3}}
    """
    def makeOverlap(row):
        listRow = json.loads(row['ingredients'])
        ingDict = {ingId : (ingName, quant) for ingId, ingName, quant in listRow}
        recipe_traits = [row['tradeskill'], row['rarity'], row['stations'], row['itemType']]
        return recipe_traits, ingDict 
    def makeLabel(row):
        listRow = json.loads(row['ingredients'])
        ingDict = {(ingId if ingId is not None else ingName) : ((ingName, quant) if ingName is not None else (ingId, quant)) for ingId, ingName, quant in listRow}
        return ingDict
        
    overlapDict = {}
    labelDict = {}
    table.apply(lambda row: overlapDict.update({row['output'] : makeOverlap(row)}), axis=1)
    table.apply(lambda row: labelDict.update({row['output'] : makeLabel(row)}), axis=1)
    return overlapDict, labelDict
overlapDict, labelDict = createOverlapDict(recipe_table)



In [120]:
overlapDict['heavyfeet_furlinedt5']

(['Armoring', 0, "['Forge Tier 5']", 'Armor'],
 {'Tier 5 Metals': (None, 6), 'Leathers': (None, 2), 'Cloth': (None, 1)})

In [76]:
labelDict

{'house_housingitem_buff_fishing_bonus_t3': {'timbert5': ('Ironwood Planks',
   25),
  'ingott5': ('Orichalcum Ingot', 20),
  'woodstaint5': ('Mahogany Stain', 1),
  'house_housingitem_fishing_veryrarefish_c1': ('Taxidermied Daemonaja', 1),
  'house_housingitem_buff_fishing_bonus_t2': ('Basic Fishing Gathering Trophy',
   1)},
 'house_housingitem_buff_harvesting_bonus_t3': {'timbert5': ('Ironwood Planks',
   25),
  'ingott5': ('Orichalcum Ingot', 20),
  'woodstaint5': ('Mahogany Stain', 1),
  'mercurialtokent1': ('Mercurial Token', 1),
  'house_housingitem_buff_harvesting_bonus_t2': ('Basic Harvesting Gathering Trophy',
   1)},
 'house_housingitem_buff_logging_bonus_t3': {'timbert5': ('Ironwood Planks',
   25),
  'ingott5': ('Orichalcum Ingot', 20),
  'woodstaint5': ('Mahogany Stain', 1),
  'pureresint1': ('Pure Resin', 1),
  'house_housingitem_buff_logging_bonus_t2': ('Basic Logging Gathering Trophy',
   1)},
 'house_housingitem_buff_luck_bonus_t3': {'timbert5': ('Ironwood Planks', 25

In [134]:
#This is the main analysis function for the data.
#It creates a dictionary of the form {streamer1: {streamer2: overlap, streamer3: overlap}}
#This allows us to have an integer of overlapping viewers for each streamer with every other streamer
# def CreateOverlapDict2(odict):
#     viewerOverlapDict = {}
#     count = 1
#     completedStreamers = [] #Save which streamers have been processed to avoid repeating
#     for key in odict:
#         odict[key] = set(odict[key]) #Make viewer list a set to dramatically decrease comparison time
#     for key in odict:
#         tempList = {}

#         totalLength = len(odict.keys())
#         print(str(count) + "/" + str(totalLength)) #Print progress so I can keep track

#         for comparisonKey in odict: #Loop through every key again for each key in the dictionary
#             if(comparisonKey != key and comparisonKey not in completedStreamers): #If its not a self comparison and the comparison hasn't already been completed
#                 overlapSize = len(odict[key] & odict[comparisonKey]) #Find the overlap size of the two streamers using set intersection
#                 if(overlapSize > 300 ):
#                     tempList[comparisonKey] = overlapSize #If the size is over 300 add {comparisonStreamer: overlap} to the dictionary
#         viewerOverlapDict[key] = tempList #Add this comparison dictionary to the larger dictionary for that streamer
#         completedStreamers.append(key) #Add the streamer to completed as no comparisons using this streamer need to be done anymore
#         count+=1
#     return viewerOverlapDict


#Generates a new csv file for the edge list on Gephi
def GenerateGephiData(odict):
    fileString = "data/%s" % (now.strftime("%m.%d.%Y.%H.%M.%SEDGELIST.csv"))
    with open(fileString, 'w') as csvfile:
        writer = csv.writer(csvfile)
        #writer.writeheader()
        writer.writerow(["Source", "Target", "Weight"]) #These column headers are used in Gephi automatically
        for key, value in odict.items():
            nodeA = key
            traits = value[0]
            ings = value[1]
            for node, recipeValue in ings.items():
                nodeB = node
                print(nodeA + " " + nodeB + " recipeValue : " + str(recipeValue[1]) + " traits: " + str(traits))
                writer.writerow([nodeB, nodeA, recipeValue[1], traits[0], traits[1], traits[2], traits[3]]) #nodeA is ingredient, nodeB is recipe output, and count is the quantity

#Generates a new csv file for the node list labels on Gephi
def GenerateGephiLabels(rawDict):
    fileString = "data/%s" % (now.strftime("%m.%d.%Y.%H.%M.%SLABELS.csv"))
    print("Generating Labels...")
    sys.stdout.flush()
    with open(fileString, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["ID", "Label", "Count"]) #These columns are used in Gephi automatically
        for key, value in rawDict.items():
            for k, v in value.items():
                print(value)
                writer.writerow([k, v[0], 1]) #This data is streamer1, streamer1, and # of unique viewers for streamer1
                
                
                

In [135]:
now = datetime.now()

GenerateGephiData(overlapDict)

house_housingitem_buff_fishing_bonus_t3 timbert5 recipeValue : 25 traits: ['Furnishing', 3, "['Workshop Tier 5']", 'HousingItem']
house_housingitem_buff_fishing_bonus_t3 ingott5 recipeValue : 20 traits: ['Furnishing', 3, "['Workshop Tier 5']", 'HousingItem']
house_housingitem_buff_fishing_bonus_t3 woodstaint5 recipeValue : 1 traits: ['Furnishing', 3, "['Workshop Tier 5']", 'HousingItem']
house_housingitem_buff_fishing_bonus_t3 house_housingitem_fishing_veryrarefish_c1 recipeValue : 1 traits: ['Furnishing', 3, "['Workshop Tier 5']", 'HousingItem']
house_housingitem_buff_fishing_bonus_t3 house_housingitem_buff_fishing_bonus_t2 recipeValue : 1 traits: ['Furnishing', 3, "['Workshop Tier 5']", 'HousingItem']
house_housingitem_buff_harvesting_bonus_t3 timbert5 recipeValue : 25 traits: ['Furnishing', 3, "['Workshop Tier 5']", 'HousingItem']
house_housingitem_buff_harvesting_bonus_t3 ingott5 recipeValue : 20 traits: ['Furnishing', 3, "['Workshop Tier 5']", 'HousingItem']
house_housingitem_buff

house_housingitem_settler_stove01_b timbert3 recipeValue : 35 traits: ['Furnishing', 0, "['Workshop Tier 3']", 'HousingItem']
house_housingitem_settler_stove01_b ingott3 recipeValue : 10 traits: ['Furnishing', 0, "['Workshop Tier 3']", 'HousingItem']
house_housingitem_settler_stove01_b woodstaint3 recipeValue : 2 traits: ['Furnishing', 0, "['Workshop Tier 3']", 'HousingItem']
house_housingitem_settler_decor_garden_windchimes01 timbert3 recipeValue : 10 traits: ['Furnishing', 2, "['Workshop Tier 3']", 'HousingItem']
house_housingitem_settler_decor_garden_windchimes01 ingott2 recipeValue : 5 traits: ['Furnishing', 2, "['Workshop Tier 3']", 'HousingItem']
house_housingitem_settler_decor_garden_windchimes01 fibert1 recipeValue : 5 traits: ['Furnishing', 2, "['Workshop Tier 3']", 'HousingItem']
dyeb111 watert1 recipeValue : 3 traits: ['Cooking', 0, "['Kitchen Tier 3']", 'Dye']
dyeb111 pigmentbrown recipeValue : 2 traits: ['Cooking', 0, "['Kitchen Tier 3']", 'Dye']
dyeb111 pigmentturquoise r

lightlegs_covenantrank1_cleric lightlegs_covenantrank1 recipeValue : 1 traits: ['Armoring', 1, "['Outfitting Station Tier 2']", 'Armor']
lighthands_covenantrank5_cleric azoth_currency recipeValue : 50 traits: ['Armoring', 3, "['Outfitting Station Tier 2']", 'Armor']
lighthands_covenantrank5_cleric covenantseal_cleric recipeValue : 1 traits: ['Armoring', 3, "['Outfitting Station Tier 2']", 'Armor']
lighthands_covenantrank5_cleric lighthands_covenantrank5 recipeValue : 1 traits: ['Armoring', 3, "['Outfitting Station Tier 2']", 'Armor']
lightlegs_covenantrank2_cleric azoth_currency recipeValue : 50 traits: ['Armoring', 2, "['Outfitting Station Tier 2']", 'Armor']
lightlegs_covenantrank2_cleric covenantseal_cleric recipeValue : 1 traits: ['Armoring', 2, "['Outfitting Station Tier 2']", 'Armor']
lightlegs_covenantrank2_cleric lightlegs_covenantrank2 recipeValue : 1 traits: ['Armoring', 2, "['Outfitting Station Tier 2']", 'Armor']
lightlegs_covenantrank3_cleric azoth_currency recipeValue : 5

mediumfeet_covenantrank5_barbarian azoth_currency recipeValue : 50 traits: ['Armoring', 3, "['Outfitting Station Tier 2']", 'Armor']
mediumfeet_covenantrank5_barbarian covenantseal_barbarian recipeValue : 1 traits: ['Armoring', 3, "['Outfitting Station Tier 2']", 'Armor']
mediumfeet_covenantrank5_barbarian mediumfeet_covenantrank5 recipeValue : 1 traits: ['Armoring', 3, "['Outfitting Station Tier 2']", 'Armor']
mediumhead_syndicaterank5_barbarian azoth_currency recipeValue : 50 traits: ['Armoring', 3, "['Outfitting Station Tier 2']", 'Armor']
mediumhead_syndicaterank5_barbarian syndicateseal_barbarian recipeValue : 1 traits: ['Armoring', 3, "['Outfitting Station Tier 2']", 'Armor']
mediumhead_syndicaterank5_barbarian mediumhead_syndicaterank5 recipeValue : 1 traits: ['Armoring', 3, "['Outfitting Station Tier 2']", 'Armor']
mediumhead_syndicaterank3_barbarian azoth_currency recipeValue : 50 traits: ['Armoring', 2, "['Outfitting Station Tier 2']", 'Armor']
mediumhead_syndicaterank3_barba

mediumhands_syndicateplaguedoctort5_brigand syndicateseal_brigand recipeValue : 1 traits: ['Armoring', 3, "['Outfitting Station Tier 2']", 'Armor']
mediumhands_syndicateplaguedoctort5_brigand mediumhands_syndicateplaguedoctort5 recipeValue : 1 traits: ['Armoring', 3, "['Outfitting Station Tier 2']", 'Armor']
mediumlegs_syndicateplaguedoctort5_brigand azoth_currency recipeValue : 50 traits: ['Armoring', 3, "['Outfitting Station Tier 2']", 'Armor']
mediumlegs_syndicateplaguedoctort5_brigand syndicateseal_brigand recipeValue : 1 traits: ['Armoring', 3, "['Outfitting Station Tier 2']", 'Armor']
mediumlegs_syndicateplaguedoctort5_brigand mediumlegs_syndicateplaguedoctort5 recipeValue : 1 traits: ['Armoring', 3, "['Outfitting Station Tier 2']", 'Armor']
mediumchest_syndicateplaguedoctort5_brigand azoth_currency recipeValue : 50 traits: ['Armoring', 3, "['Outfitting Station Tier 2']", 'Armor']
mediumchest_syndicateplaguedoctort5_brigand syndicateseal_brigand recipeValue : 1 traits: ['Armoring

malachitet2 alchemyspiritt2 recipeValue : 1 traits: ['Stonecutting', 1, "['Stonecutting Table Tier 3']", 'Resource']
malachitet2 Solvent recipeValue : 2 traits: ['Stonecutting', 1, "['Stonecutting Table Tier 3']", 'Resource']
onyxt2 onyxt1 recipeValue : 3 traits: ['Stonecutting', 1, "['Stonecutting Table Tier 3']", 'Resource']
onyxt2 alchemylifet2 recipeValue : 1 traits: ['Stonecutting', 1, "['Stonecutting Table Tier 3']", 'Resource']
onyxt2 Solvent recipeValue : 2 traits: ['Stonecutting', 1, "['Stonecutting Table Tier 3']", 'Resource']
opalt2 opalt1 recipeValue : 3 traits: ['Stonecutting', 1, "['Stonecutting Table Tier 3']", 'Resource']
opalt2 alchemyspiritt2 recipeValue : 1 traits: ['Stonecutting', 1, "['Stonecutting Table Tier 3']", 'Resource']
opalt2 Solvent recipeValue : 2 traits: ['Stonecutting', 1, "['Stonecutting Table Tier 3']", 'Resource']
rubyt2 rubyt1 recipeValue : 3 traits: ['Stonecutting', 1, "['Stonecutting Table Tier 3']", 'Resource']
rubyt2 alchemyfiret2 recipeValue : 

1hswordbt4 Leathers recipeValue : 1 traits: ['Weaponsmithing', 0, "['Forge Tier 4']", 'Weapon']
2hgreataxet4 Tier 4 Metals recipeValue : 14 traits: ['Weaponsmithing', 0, "['Forge Tier 4']", 'Weapon']
2hgreataxet4 Refined Wood recipeValue : 3 traits: ['Weaponsmithing', 0, "['Forge Tier 4']", 'Weapon']
2hgreataxet4 Leathers recipeValue : 2 traits: ['Weaponsmithing', 0, "['Forge Tier 4']", 'Weapon']
2hwarhammert4 Tier 4 Metals recipeValue : 14 traits: ['Weaponsmithing', 0, "['Forge Tier 4']", 'Weapon']
2hwarhammert4 Refined Wood recipeValue : 3 traits: ['Weaponsmithing', 0, "['Forge Tier 4']", 'Weapon']
2hwarhammert4 Leathers recipeValue : 2 traits: ['Weaponsmithing', 0, "['Forge Tier 4']", 'Weapon']
1hshieldat4 Tier 4 Metals recipeValue : 9 traits: ['Weaponsmithing', 0, "['Forge Tier 4']", 'Weapon']
1hshieldat4 Refined Wood recipeValue : 2 traits: ['Weaponsmithing', 0, "['Forge Tier 4']", 'Weapon']
1hshieldat4 Leathers recipeValue : 1 traits: ['Weaponsmithing', 0, "['Forge Tier 4']", 'We

2hspearcrafteddungeon7t5 Refined Wood recipeValue : 3 traits: ['Engineering', 0, "['Forge Tier 5']", 'Weapon']
2hspearcrafteddungeon7t5 Leathers recipeValue : 2 traits: ['Engineering', 0, "['Forge Tier 5']", 'Weapon']
heavychest_crafteddungeon7t5 stickyvinest1 recipeValue : 10 traits: ['Armoring', 0, "['Forge Tier 5']", 'Armor']
heavychest_crafteddungeon7t5 Tier 5 Metals recipeValue : 16 traits: ['Armoring', 0, "['Forge Tier 5']", 'Armor']
heavychest_crafteddungeon7t5 Leathers recipeValue : 12 traits: ['Armoring', 0, "['Forge Tier 5']", 'Armor']
heavychest_crafteddungeon7t5 Cloth recipeValue : 7 traits: ['Armoring', 0, "['Forge Tier 5']", 'Armor']
heavyhands_crafteddungeon6t5 spectraldustt1 recipeValue : 10 traits: ['Armoring', 0, "['Forge Tier 5']", 'Armor']
heavyhands_crafteddungeon6t5 Tier 5 Metals recipeValue : 4 traits: ['Armoring', 0, "['Forge Tier 5']", 'Armor']
heavyhands_crafteddungeon6t5 Leathers recipeValue : 4 traits: ['Armoring', 0, "['Forge Tier 5']", 'Armor']
heavyhands_

In [103]:
GenerateGephiLabels(labelDict)

Generating Labels...
{'timbert5': ('Ironwood Planks', 25), 'ingott5': ('Orichalcum Ingot', 20), 'woodstaint5': ('Mahogany Stain', 1), 'house_housingitem_fishing_veryrarefish_c1': ('Taxidermied Daemonaja', 1), 'house_housingitem_buff_fishing_bonus_t2': ('Basic Fishing Gathering Trophy', 1)}
{'timbert5': ('Ironwood Planks', 25), 'ingott5': ('Orichalcum Ingot', 20), 'woodstaint5': ('Mahogany Stain', 1), 'house_housingitem_fishing_veryrarefish_c1': ('Taxidermied Daemonaja', 1), 'house_housingitem_buff_fishing_bonus_t2': ('Basic Fishing Gathering Trophy', 1)}
{'timbert5': ('Ironwood Planks', 25), 'ingott5': ('Orichalcum Ingot', 20), 'woodstaint5': ('Mahogany Stain', 1), 'house_housingitem_fishing_veryrarefish_c1': ('Taxidermied Daemonaja', 1), 'house_housingitem_buff_fishing_bonus_t2': ('Basic Fishing Gathering Trophy', 1)}
{'timbert5': ('Ironwood Planks', 25), 'ingott5': ('Orichalcum Ingot', 20), 'woodstaint5': ('Mahogany Stain', 1), 'house_housingitem_fishing_veryrarefish_c1': ('Taxidermi

{'Tier 4 Cloth': ('Tier 4 Cloth', 12), 'Leathers': ('Leathers', 4), 'Metals': ('Metals', 2)}
{'Tier 4 Cloth': ('Tier 4 Cloth', 12), 'Leathers': ('Leathers', 4), 'Metals': ('Metals', 2)}
{'Tier 4 Cloth': ('Tier 4 Cloth', 6), 'Leathers': ('Leathers', 2), 'Metals': ('Metals', 1)}
{'Tier 4 Cloth': ('Tier 4 Cloth', 6), 'Leathers': ('Leathers', 2), 'Metals': ('Metals', 1)}
{'Tier 4 Cloth': ('Tier 4 Cloth', 6), 'Leathers': ('Leathers', 2), 'Metals': ('Metals', 1)}
{'Tier 4 Cloth': ('Tier 4 Cloth', 11), 'Leathers': ('Leathers', 6), 'Metals': ('Metals', 1)}
{'Tier 4 Cloth': ('Tier 4 Cloth', 11), 'Leathers': ('Leathers', 6), 'Metals': ('Metals', 1)}
{'Tier 4 Cloth': ('Tier 4 Cloth', 11), 'Leathers': ('Leathers', 6), 'Metals': ('Metals', 1)}
{'emeraldcutt3': ('Cut Brilliant Emerald', 1), 'Trinket Bands': ('Trinket Bands', 1), 'Trinket Gem Settings': ('Trinket Gem Settings', 1), 'Precious Metals': ('Precious Metals', 1)}
{'emeraldcutt3': ('Cut Brilliant Emerald', 1), 'Trinket Bands': ('Trinket Ban

{'workorder_faction_provisions01_t1': ('Cooking Recipe', 1), 'porkt1': ('Pork', 3), 'saltt1': ('Salt', 3)}
{'watert1': ('Water', 1), 'Tier 2 Medicinal Reagents': ('Tier 2 Medicinal Reagents', 1)}
{'watert1': ('Water', 1), 'Tier 2 Medicinal Reagents': ('Tier 2 Medicinal Reagents', 1)}
{'Tier 2 Raw Foods': ('Tier 2 Raw Foods', 1), 'Raw Foods': ('Raw Foods', 1)}
{'Tier 2 Raw Foods': ('Tier 2 Raw Foods', 1), 'Raw Foods': ('Raw Foods', 1)}
{'Tier 1 Raw Foods': ('Tier 1 Raw Foods', 1)}
{'watert1': ('Water', 1), 'Tier 2 Magical Reagents': ('Tier 2 Magical Reagents', 1)}
{'watert1': ('Water', 1), 'Tier 2 Magical Reagents': ('Tier 2 Magical Reagents', 1)}
{'cranberryt1': ('Cranberry', 1), 'sugart1': ('Sugar', 1), 'watert1': ('Water', 1)}
{'cranberryt1': ('Cranberry', 1), 'sugart1': ('Sugar', 1), 'watert1': ('Water', 1)}
{'cranberryt1': ('Cranberry', 1), 'sugart1': ('Sugar', 1), 'watert1': ('Water', 1)}
{'Tier 2 Raw Foods': ('Tier 2 Raw Foods', 1), 'Raw Foods': ('Raw Foods', 1)}
{'Tier 2 Raw Foo

{'azoth_currency': ('azoth_currency', 50), 'syndicateseal_occultist': ('Syndicate Occultist Seal', 1), 'heavyhands_syndicaterank2': ('Reinforced Syndicate Scrivener Gauntlets of the Barbarian', 1)}
{'azoth_currency': ('azoth_currency', 50), 'syndicateseal_occultist': ('Syndicate Occultist Seal', 1), 'heavyhands_syndicaterank1': ('Reinforced Syndicate Adept Gauntlets of the Barbarian', 1)}
{'azoth_currency': ('azoth_currency', 50), 'syndicateseal_occultist': ('Syndicate Occultist Seal', 1), 'heavyhands_syndicaterank1': ('Reinforced Syndicate Adept Gauntlets of the Barbarian', 1)}
{'azoth_currency': ('azoth_currency', 50), 'syndicateseal_occultist': ('Syndicate Occultist Seal', 1), 'heavyhands_syndicaterank1': ('Reinforced Syndicate Adept Gauntlets of the Barbarian', 1)}
{'azoth_currency': ('azoth_currency', 50), 'covenantseal_brigand': ('Covenant Brigand Seal', 1), 'heavyhead_covenantinquisitort5': ('Reinforced Covenant Inquisitor Hat of the Warden', 1)}
{'azoth_currency': ('azoth_curre

{'azoth_currency': ('azoth_currency', 50), 'syndicateseal_barbarian': ('Syndicate Barbarian Seal', 1), 'mediumfeet_syndicaterank4': ('Reinforced Syndicate Cabalist Shoes of the Brigand', 1)}
{'azoth_currency': ('azoth_currency', 50), 'syndicateseal_barbarian': ('Syndicate Barbarian Seal', 1), 'mediumfeet_syndicaterank4': ('Reinforced Syndicate Cabalist Shoes of the Brigand', 1)}
{'azoth_currency': ('azoth_currency', 50), 'syndicateseal_barbarian': ('Syndicate Barbarian Seal', 1), 'mediumfeet_syndicaterank3': ('Reinforced Syndicate Chronicler Shoes of the Brigand', 1)}
{'azoth_currency': ('azoth_currency', 50), 'syndicateseal_barbarian': ('Syndicate Barbarian Seal', 1), 'mediumfeet_syndicaterank3': ('Reinforced Syndicate Chronicler Shoes of the Brigand', 1)}
{'azoth_currency': ('azoth_currency', 50), 'syndicateseal_barbarian': ('Syndicate Barbarian Seal', 1), 'mediumfeet_syndicaterank3': ('Reinforced Syndicate Chronicler Shoes of the Brigand', 1)}
{'azoth_currency': ('azoth_currency', 5

{'workorder_1310_1314_l40bow_huntress': ('Huntress Work Order', 1), '1310_1314_ancientwood': ('Ancient Wood', 14), '1310_1314_wyrdwoodresin': ('Wyrdwood Resin', 15), '1310_1314_hawkfeathers': ('Hawk Feather', 16), '1310_1314_luckyrabbitsfoot': ("Lucky Rabbit's Foot", 1)}
{'workorder_1310_1314_l40bow_huntress': ('Huntress Work Order', 1), '1310_1314_ancientwood': ('Ancient Wood', 14), '1310_1314_wyrdwoodresin': ('Wyrdwood Resin', 15), '1310_1314_hawkfeathers': ('Hawk Feather', 16), '1310_1314_luckyrabbitsfoot': ("Lucky Rabbit's Foot", 1)}
{'workorder_1310_1314_l40bow_huntress': ('Huntress Work Order', 1), '1310_1314_ancientwood': ('Ancient Wood', 14), '1310_1314_wyrdwoodresin': ('Wyrdwood Resin', 15), '1310_1314_hawkfeathers': ('Hawk Feather', 16), '1310_1314_luckyrabbitsfoot': ("Lucky Rabbit's Foot", 1)}
{'cornt1': ('Corn', 1), 'buttert1': ('Butter', 1), 'saltt1': ('Salt', 1)}
{'cornt1': ('Corn', 1), 'buttert1': ('Butter', 1), 'saltt1': ('Salt', 1)}
{'cornt1': ('Corn', 1), 'buttert1': 

{'watert1': ('Water', 1), 'Tier 4 Magical Reagents': ('Tier 4 Magical Reagents', 1), 'Air Reagents': ('Air Reagents', 1), 'Spirit Reagents': ('Spirit Reagents', 1)}
{'watert1': ('Water', 1), 'Tier 4 Magical Reagents': ('Tier 4 Magical Reagents', 1), 'Air Reagents': ('Air Reagents', 1), 'Spirit Reagents': ('Spirit Reagents', 1)}
{'watert1': ('Water', 1), 'Tier 4 Magical Reagents': ('Tier 4 Magical Reagents', 1), 'Air Reagents': ('Air Reagents', 1), 'Spirit Reagents': ('Spirit Reagents', 1)}
{'watert1': ('Water', 1), 'Tier 4 Magical Reagents': ('Tier 4 Magical Reagents', 1), 'Air Reagents': ('Air Reagents', 1), 'Spirit Reagents': ('Spirit Reagents', 1)}
{'melont1': ('Melon', 1), 'berryt1': ('Berry', 1), 'nutt1': ('Nut', 1), 'honeyt1': ('Honey', 1), 'blueberryt1': ('Blueberry', 1), 'cinnamont1': ('Cinnamon', 1)}
{'melont1': ('Melon', 1), 'berryt1': ('Berry', 1), 'nutt1': ('Nut', 1), 'honeyt1': ('Honey', 1), 'blueberryt1': ('Blueberry', 1), 'cinnamont1': ('Cinnamon', 1)}
{'melont1': ('Melo

{'timbert4': ('Wyrdwood Planks', 25), 'ingott4': ('Starmetal Ingot', 20), 'woodstaint4': ('Oak Stain', 1), 'quartermastersnotest1': ("Quartermaster's Notes", 1), 'house_housingitem_buff_blacksmithing_bonus_t1': ('Minor Weaponsmithing Crafting Trophy', 1)}
{'timbert4': ('Wyrdwood Planks', 25), 'ingott4': ('Starmetal Ingot', 20), 'woodstaint4': ('Oak Stain', 1), 'quartermastersnotest1': ("Quartermaster's Notes", 1), 'house_housingitem_buff_blacksmithing_bonus_t1': ('Minor Weaponsmithing Crafting Trophy', 1)}
{'timbert4': ('Wyrdwood Planks', 25), 'ingott4': ('Starmetal Ingot', 20), 'woodstaint4': ('Oak Stain', 1), 'barkflesht1': ('Barkflesh', 1), 'house_housingitem_buff_damage_vs_angryearth_t1': ('Minor Angry Earth Combat Trophy', 1)}
{'timbert4': ('Wyrdwood Planks', 25), 'ingott4': ('Starmetal Ingot', 20), 'woodstaint4': ('Oak Stain', 1), 'barkflesht1': ('Barkflesh', 1), 'house_housingitem_buff_damage_vs_angryearth_t1': ('Minor Angry Earth Combat Trophy', 1)}
{'timbert4': ('Wyrdwood Plan